In [1]:
import torch.nn as nn 
import torch.nn.functional as F
import torch

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        
        self.layers+=[nn.Conv2d(3, 16,  kernel_size=3) , 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 16,  kernel_size=3, stride=2), 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 32,  kernel_size=3), 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(32, 32,  kernel_size=3, stride=2), 
                      nn.ReLU(inplace=True)]
        self.fc = nn.Linear(32*5*5, 10)
    def forward(self, x):
        for i in range(len(self.layers)):
          x = self.layers[i](x)
        x = x.view(-1, 32*5*5)
        x = self.fc(x)
        return x



In [2]:
from lib.eugene import train, test
from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset

  
from torchvision import datasets, transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
# Extract a subset of 100 (class balanced) samples per class

accs = []

for seed in [1,2]:
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 5000))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:10]] for classe in range(0, 10)])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[10:210]] for classe in range(0, 10)])


    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)


    model = Net()
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=epoch%5==0)

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.303445
Train Epoch: 5 [0/100 (0%)]	Loss: 2.301973
Train Epoch: 10 [0/100 (0%)]	Loss: 2.298800
Train Epoch: 15 [0/100 (0%)]	Loss: 2.293989
Train Epoch: 20 [0/100 (0%)]	Loss: 2.285856
Train Epoch: 25 [0/100 (0%)]	Loss: 2.269397
Train Epoch: 30 [0/100 (0%)]	Loss: 2.234462
Train Epoch: 35 [0/100 (0%)]	Loss: 2.195818
Train Epoch: 40 [0/100 (0%)]	Loss: 2.127816
Train Epoch: 45 [0/100 (0%)]	Loss: 2.038943
Train Epoch: 50 [0/100 (0%)]	Loss: 1.900288
Train Epoch: 55 [0/100 (0%)]	Loss: 1.759894
Train Epoch: 60 [0/100 (0%)]	Loss: 1.567394
Train Epoch: 65 [0/100 (0%)]	Loss: 1.397699
Train Epoch: 70 [0/100 (0%)]	Loss: 1.216336
Train Epoch: 75 [0/100 (0%)]	Loss: 1.004221
Train Epoch: 80 [0/100 (0%)]	Loss: 1.125036
Train Epoch: 85 [0/100 (0%)]	Loss: 1.165849
Train Epoch: 90 [0/100 (0%)]	Loss: 0.715333
Train Epoch: 95 [0/100 (0%)]	Loss: 0

/home/benjamin/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 10.6894, Accuracy: 375/2000 (18.75%)

Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.303427
Train Epoch: 5 [0/100 (0%)]	Loss: 2.302537
Train Epoch: 10 [0/100 (0%)]	Loss: 2.300617
Train Epoch: 15 [0/100 (0%)]	Loss: 2.298052
Train Epoch: 20 [0/100 (0%)]	Loss: 2.294733
Train Epoch: 25 [0/100 (0%)]	Loss: 2.289792
Train Epoch: 30 [0/100 (0%)]	Loss: 2.281243
Train Epoch: 35 [0/100 (0%)]	Loss: 2.264101
Train Epoch: 40 [0/100 (0%)]	Loss: 2.227575
Train Epoch: 45 [0/100 (0%)]	Loss: 2.174521
Train Epoch: 50 [0/100 (0%)]	Loss: 2.107082
Train Epoch: 55 [0/100 (0%)]	Loss: 2.027960
Train Epoch: 60 [0/100 (0%)]	Loss: 1.932490
Train Epoch: 65 [0/100 (0%)]	Loss: 1.834360
Train Epoch: 70 [0/100 (0%)]	Loss: 1.699047
Train Epoch: 75 [0/100 (0%)]	Loss: 1.513784
Train Epoch: 80 [0/100 (0%)]	Loss: 1.322633
Train Epoch: 85 [0/100 (0%)]	Loss: 1.699984
Train Epoch: 90 [0/100 (0%)]	Loss: 1.835116
Train Epoch: 95 [0/100 (0%)]	Loss: 1.647890

Test